# Sistemas de Recomendación (1)

En esta serie de notebooks vamos a explorar algunas técnicas tradicionales de recomendación como:

*   Recomendación no personalizada y estereotipada (basada en popularidad).
*   Recomendación basada en contenido.
*   Recomendación basada en filtrado colaborativo.
*   Recomendación basada en factorización de matrices.

### Objetivos

Implementar algunos sistemas de recomendación simples para entender su funcionamiento utilizando un dataset de películas.

## Repasemos... qué son los sistemas de recomedación?

Los sistemas de recomendación se encuentran en la actualidad entre las aplicaciones de Data Science más populares. Son algoritmos "simples" que intentan proveer información relevante a los usuarios a partir de descubrir patrones en los datasets. Por ejemplo, la frase típica que aparece en las tiendas virtuales "Quienes compraron esto, también compraron..."

<p align="center">
<img src="https://github.com/caiomiyashiro/RecommenderSystemsNotebooks/raw/359c2f549cec31842e14bee5a3dfdd252817f32f/images/notebook1_image1.jpeg" alt="Naïve recommendation" style="width: 400px;"/>
</p>

El uso masivo de Internet fue uno de los factores que resaltó la importancia de los sistemas de recomendación. Debido a la sobrecarga de información que "sufrimos" a diario, puede ser dificil encontrar (o incluso definir) aquello que se quiere o desea. En contrapartida, la gran cantidad de datos e información disponible permitió la creación de plataformas para su análsis y permitir proveer a los usuarios solo la información que pueda resultarles relevante.

## Manos a la obra!

Para el propósito de este notebook, vamos a trabajar con colecciones de películas y sus ratings. Vamos a utilizar un [dataset](http://files.grouplens.org/datasets/movielens/ml-100k.zip) creado por la *University of Minnesota* que contiene 100k ratings pertenecientes a 1k usuarios sobre 1.7k películas.

## Recomendaciones no personalizadas

La forma más básica de recomendación es la no-personalizada, que no considera ni las preferencias de los individuos ni su contexto para la realización de las recomendaciones. 

Por ejemplo, pensemos en un usuario nuevo de Amazon (o su plataforma favorita de ventas por internet). Cuando recién cree su perfil no va a haber comprado ningún artículo, entonce Amazon no va a poder conocer cuales son los intereses o gustos de este nuevo usuario. Entonces, la mejor manera para realizar las primeras recomendaciones a este usuario es mostrarle lo que otros usuarios hayan comprado, independientemente de sus gustos individuales. 







Lo primero que vamos a hacer es importar las bibliotecas necesarias:

In [0]:
import pandas as pd 
import numpy as np

A continuación, leeremos los datos utilizando `read_csv`. Las columnas del dataset se encuentra separado por `\t`, de modo que debemos pasarlo como parámetro. Finalmente, también pasamos por parámetro los nombres de las columnas.


In [0]:
url = 'https://raw.githubusercontent.com/tommantonela/sistemasRecomendacion2019/master/ml-100k/u.data'
df = pd.read_csv(url, sep='\t', names=['user_id','movie_id','rating','timestamp'])

Vamos a imprimir parte de los datos para ver con qué estamos tratando.

In [0]:
df.head()

Cada una de las filas representa el rating que un usuario le dió a una película. Entonces:

*   `user_id`. Id del usuario asignando el rating a la película.
*   `movie_id`. Id de la película a la que el `user_id` asignó el rating. 
*   `rating`. En escala de 1 a 5.
*   `timestamp`. Momento temporal en el que el rating fue asignado.

 Vamos a asociarle a cada id su título correspondiente. Para ello, cargamos el archivo que contiene el mapping entre los `ids` y los `movie_title` (y los géneros a los que pertenecen las películas).

In [0]:
url = 'https://raw.githubusercontent.com/tommantonela/sistemasRecomendacion2019/master/ml-100k/u.item'
movie_info = pd.read_csv(url,sep='|', encoding='latin-1', header=None, names=['movie_id','movie_title','release_date','movie_release_date',
                                                                              'IMDb url','unknown','Action','Adventure','Animation','Children','Comedy',
                                                                              'Crime','Documentary','Drama','Fantasy','Film-Noir','Horror','Musical',
                                                                              'Mystery','Romance','Sci-Fi','Thriller','War','Western'])

movie_info = movie_info.drop('movie_release_date', axis=1) # eliminamos la columna movie_release_date que es NaN para todos los registros
movie_info = movie_info.drop('IMDb url', axis=1) # eliminamos esta columna que no aporta ninguna información relevante
movie_info.head()

Luego,  mergeamos ambos `data frames`.

In [0]:
df = pd.merge(df, movie_info, on='movie_id')
df.head()

Vamos a visualizar algunas estadísticas del dataset.

In [0]:
df.describe()

Y ahora el histograma de los ratings.

In [0]:
df['rating'].hist(bins=5)

La recomendación no personalizada generalmente se encuentra basada en el cálculo de alguna métrica que retorna un valor para cada elemento que luego permite ordenarlos. Por ejemplo:

*   ***Rating promedio***. Cuál es el rating promedio de un elemento particular.
*   ***Cantidad de ratings***. Una medida de popularidad. Cuántos usuarios ratiaron un elemento? (ya sea para bien o para mal).
*    ***Porcentaje de buenos ratings***. Dado un *threshold* (por ejemplo, 4), qué porcentaje de usuarios dieron un buen rating al elemento?
*    ***Asociación***. Dados todos los usuarios que ratearon el elemento A, qué porcentaje de ellos también ratearon el elemento B?
*    ***Pearson Correlation***. Dado el rating que un usuario le dió al elemento A, se correlaciona con el rating que la misma persona le dió al elemento B?

Vamos a crear dos funciones. Primero, una función que reciba todos los valores de la métrica `column` calculado y retorne el los mejores `n` elementos. Segundo, una función que nos permita graficar los resultados.


In [0]:
def return_best_n(statistics, n,column):
    # statistics: array of size review.shape[1] containing one statistic calculated from the dataset
    # n: number of indices to be returned
    # returns: array of size *n* containing the indices of the best scored statistics
    statistics = pd.DataFrame(statistics)
    #statistics.head()
    return statistics.sort_values(by=column,axis=0, ascending = False).iloc[:n]

import matplotlib.pyplot as plt # la vamos a usar para graficar
import seaborn as sns

def graphics(statistics):
    plt.figure(figsize=(16,6))
    ax = sns.barplot(x=statistics['rating'], y=statistics['movie_title'], data=statistics, palette='deep')
    plt.title('Movie Ranking', weight='bold')
    plt.xlabel('Score', weight='bold')
    plt.ylabel('Movie Title', weight='bold')

Ahora, vamos a ver los resultados para distintas métricas.

#### Cantidad de ratings

In [0]:
ratings = pd.DataFrame(df.groupby('movie_title',as_index=True)['rating'].count().reset_index())
best = return_best_n(ratings,10,'rating')

graphics(best)

#### Rating promedio

Independientemente de la cantidad de ratings que hayan recibido las películas, cual es el promedio de esos rankings?


In [0]:
ratings = pd.DataFrame(df.groupby('movie_title')['rating'].mean())
best = return_best_n(ratings,10,'rating')
best.head()

En los resultados vemos que hay películas que tienen ratings promedio perfectos. Vamos a ver, para la película mejor rankeada cuántos ratings tuvo.

In [0]:
df[df['movie_title']==' '] # TODO: Completar con el título de la película mejor rankeada.

Aquellas películas que cuentan con un único rating no resultan muy útiles para la recomendación. 
Vamos a eliminar todas las películas que tengan menos reviews que el $25\%$ de las películas, es decir, el primer cuartil.

In [0]:
counts = pd.DataFrame(df.groupby('movie_title',as_index=True)['rating'].count().reset_index())

first_quantile = counts['rating'].quantile(0.25)

titles = counts[counts['rating'] > first_quantile]['movie_title'] #acá nos quedamos con los títulos de las pelis con una cantidad de reviews mayores al 1er quartil

df = df[df['movie_title'].isin(titles)]

df.shape


Ahora, volvemos a calcular el rating promedio:

In [0]:
ratings = pd.DataFrame(df.groupby('movie_title',as_index=True)['rating'].mean().reset_index())
best = return_best_n(ratings,10,'rating')
best

In [0]:
graphics(best)

#### Porcentaje de buenas reviews (>= 4)

Qué tenemos que hacer?

1.   Filtrar reviews con puntaje mayor o igual que 4.
2.   Dividir por el total de las reviews.

Tip: Resumen de las [funciones de agregación](https://pandas.pydata.org/pandas-docs/stable/getting_started/basics.html#descriptive-statistics)

In [0]:
counts = pd.DataFrame(df.groupby('movie_title',as_index=True)['rating'].count().reset_index()) # como antes, contamos cuantos ratings tienen la películas

filtered = df[df['rating'] > 4]  # filtramos las reviews con puntaje mayor a 4
filtered = pd.DataFrame(filtered.groupby('movie_title',as_index=True)['rating'].count().reset_index()) # calculamos cuantos ratings > 4 tienen las películas

merged = pd.merge(filtered, counts, on='movie_title') # mergeamos ambos data frames

merged.head()


In [0]:
merged['rating'] = merged['rating_x'] / merged['rating_y']
best = return_best_n(merged,10,'rating')
best

In [0]:
graphics(best)

#### Usando la métrica de IMDb

IMDb tuvo (todavía tiene?) su propia métrica para el agregado de los ratings de las películas, que se define como":

<p align="center">
<img src="https://miro.medium.com/max/552/1*fGziZl2Do-VyQXSCPq_Y2Q.png" alt="Weighted Rating" style="width: 400px;"/>
</p>

Qué tenemos que hacer?

1.   Calcular los ratings promedios de las películas (ya lo hicimos antes).
2.   Calcular la cantidad de ratings de las películas (ya lo hicimos antes).
3.   Calcular la cantidad mínima de ratings para estar en el Top de películas. En este caso como la cantidad de votos recibida por el 70% de las películas. (ya hicimos algo parecido).
4.    Calcular el rating promedio general (ya hicimos algo parecido).
5.    Calcular W a partir de todos los otros valores (ya hicimos algo parecido).



In [0]:
# TODO!
averages = # completar
counts = # completar
minimum = # completar
general_average = # completar

merged = pd.merge(averages, counts, on='movie_title')
merged['IMDb Metric'] = (merged['rating_x']*merged['rating_y'] + general_average*minimum)/(merged['rating_y'] + minimum)

best = return_best_n(merged,10,'IMDb Metric')
best

#### Pearson Correlation

El análisis de correlaciones evalúa si los ratings que un usuario da a un elemento A puede dar "pistas" respecto al rating que daría al elemento B. 

Una correlación cercada a $1$ indica que los usuarios tienen a darle a ambas películas involucradas ratings similares. En este caso, se permiten realizar recomendaciones del tipo "Como te gustó X, te podría gustar Y".

Vamos a calcular la correlación del rating medio de las películas respecto a la película con la promedio de ratings en la mediana (es decir, la película cuyo rating promedio se encuentra en el cuartil $0.5$).


In [0]:
averages = pd.DataFrame(df.groupby('movie_title',as_index=True)['rating'].mean().reset_index())
mediana = averages.quantile(0.5)

median_movie = averages[averages['rating'] == mediana[0]]
median_movie = median_movie.iloc[0]['movie_title'] # en caso de que haya más de una, nos quedamos con la primera que retornó
median_movie

In [0]:
# recordemos que df tiene las películas con más reviews que el 25% de ellas

# para calcular la correlación necesitamos una matriz de películas x ratings, donde las columnas representen a cada una de las películas
movie_matrix = df.pivot_table(index='user_id', columns='movie_title', values='rating') 

movie_matrix.head()

In [0]:
# nos quedamos con la columna de los ratings de la película que queremos
median_movie_ratings = movie_matrix[median_movie]

# calculamos la correlación de toda la matriz con la columna seleccionada. El resultado es una matriz de películas x películas
correlation = movie_matrix.corrwith(median_movie_ratings)

correlation.sort_values(ascending=False) #ordenamos

corr_contact = pd.DataFrame(correlation, columns=['Correlation']) #pasamos a un data frame
corr_contact.dropna(inplace=True) # sacamos los NaN
corr_contact.head()


## Recomendación estereotipada

Una pequeña "mejora" que se puede hacer a la recomendación no personalizada es la recomendación estereotipada. Por ejemplo, los rankings por sexo/género, edad o ciudad, pueden mejorar la calidad de las recomendaciones si se cree que entre los elementos a recomendar existe realmente una diferenciación de acuerdo a las características de los diferentes segmentos.

Qué pasa si tenemos en cuenta información demográfica para hacer las recomendaciones? Por ejemplo, varían las recomendaciones si consideramos el sexo/género de quienes ratearon las películas?

Primero, tenemos que cargar la información de los usuarios y mergearla en la tabla de reviews.

In [0]:
url = 'https://raw.githubusercontent.com/tommantonela/sistemasRecomendacion2019/master/ml-100k/u.user'
user_info = pd.read_csv(url, sep='|', names=['user_id','age','gender','occupation','zip_code'])

user_info.head()

In [0]:
df = df.merge(user_info, on='user_id',how='inner')
df.head()

In [0]:
# Películas mejor rankeadas para mujeres
averages_F = pd.DataFrame(df[df['gender'] == 'F'].groupby('movie_title',as_index=True)['rating'].mean().reset_index())
best = return_best_n(averages_F,10,'rating')
best

In [0]:
graphics(best)

In [0]:
# Películas mejor rankeadas para hombres TODO!
averages_M = # completar
best = # completar
best

#### Diferencia de rating promedio

En qué películas los hombres y mujeres tienen mayores diferencias para calificar?


In [0]:
merged = pd.merge(averages_F, averages_M, on='movie_title', suffixes=['_F',"_M"])

merged['f-m'] = merged['rating_F'] - merged['rating_M']

highest = return_best_n(merged,10,'f-m')
highest


In [0]:
# y al revés? 
# completar

## Ventajas y desventajas de la recomendación no personalizada

### Ventajas

*   No se necesita tener información previa de los usuarios respecto a sus gustos.
*   Las estadísticas calculadas con simple y fácilmente explicables.

### Desventajas

*    Para poder proveer buenas estadísticas, los elementos deben tener una cantidad razonable de reviews, lo que también implica tener una cantidad razonable de usuarios.
*    Al tratar a todos los usuarios de la misma forma, no es posible la identificación de grupos de usuarios (no referido solo a grupos demográficos).
*    Las recomendaciones estereotipadas solo serán útiles si existen productos que fueron específicamente diseñados para dichos grupos.


## Peeero, un problema salvaje aparece!

Como vimos estas recomendaciones simples se encuentran basadas en *scores* que pueden ser calculadas a partir de indicadores básicos:

*    *Explícitos*. Evaluaciones provistas de forma directa por los usuarios, por ejemplo, ratings, scores, likes, ...
*    *Implícitos*. Derivadas del comportamiento implícito del usuario como por ejemplo clicks, tiempo en una página, accesos, ...

The most basic way to provide recommendations is a non-personalised one que no tome en ceunta las preferencias individuales de los usuarios. 

Las métricas ***explícitas*** son "buenas" porque se puede preguntar de forma directa al usuario qué es lo que piensa. Sin embargo, algunos problemas aparecen.

1.   Cómo los usuarios saben como reflejar sus gustos en un valor numérico?
2.   La definición de los ratings varía de usuario a usuario. Por ejemplo, algunos usuarios pueden ratear la película con un 5 cuando es lo que esperaba, mientras que para otros eso representa un rating de 3 y solo califican con 5 a aquellas películas que superaron sus expectativas.
3.   La definición de los ratings varía a lo largo del tiempo. Un usuario puede dar dos ratings distintos para el mismo elemento en distintos momemntos temporales.

Por otra parte, las ***implícitas*** asumen que las acciones de los usuarios "dicen más que su palabras". Asimismo, recolectar comportamiento es más rápido que esperar a que el usuario asigne ratings y se encuentra ajeno al juicio humano inherente a las métricas explícitas. Sin embargo, tratar con las múltiples variables derivadas puede no ser sencillo, debiendo definir adecuadamente los pesos a considerar de acuerdo al dominio.

### Rating con estrellas

Uno de los tipos principales de ratings que se utiliza hoy en día se encuentra basado en los ratings de 5 estrellas. Utilizado por empresas como Amazon, anteriormente por Netflix y TripAdvisor, estas empresas utilizan estas evaluaciones para trazar un perfil entre los usuarios y sus productos, para realizar recomendaciones no personalizadas y personalizadas.

Pero, *qué significa asignar 1 o 5 estrellas?*

Las dificultades comienzan cuando la escala no se encuentra configurada o definida correctamente. Por supuesto, existen límites para ello, pero generalmente no se  proporcionan referencias concretas en las que basar la calificación. Este escenario trae inestabilidad a las distribuciones de calificaciones y empeora debido a la percepción humana, que tiende a asignar ratings de forma instantánea.

*Evaluación contextual y temporal*

El gusto y los intereses de los usuarios puede variar mucho según el contexto. En este contexto, utilizar funciones agregadas de los ratings puede involucrar el cálculo sobre ratings provistos en distintos contexto, lo que puede dar lugar a recomendaciones inestables y no confiables.

[Fischhoff, 1991](https://www.cmu.edu/dietrich/sds/docs/fischhoff/ValueElictationAnythngThere.pdf) estudiaron qué es lo uqe los usuarios tienen en cuenta al momento de asignar los ratings, encontrando que los mismos son asignados de forma instantánea, sin consierar factores históricos como el gusto personal o las características del nuevo elemento en relación a los elementos previos. Simplemente, se trata de *me gustó* o *no me gustó*. El problema con esta situación es que los ratings que asignan los usuarios a los elementos pueden variar si vuelven a ser asignados, lo que agrega más ruido a los ratings.

### Efecto de la variabilidad de los ratings

Supóngase que se cuenta con un elemento que tiene un rating promedio de $4.8$. Ahora, un nuevo usuario le asigna un rating de $1$. En este contexto, el nuevo rating será: 

$$4.8 - \frac{4.8 - 1}{100} = 4.762$$  

Ahora, cuál será el rating si en lugar de aisgnar un $1$, se hubiese asignado un $5$?

$$4.8 - \frac{4.8 - 5}{100} = 4.802$$

Nótese que el efecto de un mal ranking es mayor que el efecto de un buen ránking. El problema es que el promedio asigna un mayor peso a los scores que se encuetran más lejanos al promedio. Entonces, como el promedio que teníamos estaba más cercano al $5$ que al $1$, el $1$ tiene un mayor impacto en el promedio final.

Pero, qué pasaría si se quisiera recuperar el rating promedio que se tenía antes del $1$? Es decir, cuántos ratings de $5$ son necesarios para volver al promedio anterior?


In [0]:
rating_promedio_actual = 4.762
count = 0
while(rating_promedio_actual <= 4.8):
    rating_promedio_actual = rating_promedio_actual - ((rating_promedio_actual - 5)/100)
    print('Nuevo Rating: ' + str(rating_promedio_actual))
    count += 1
print('Se necesitaron ' + str(count) + ' ratings para volver al promedio original.')

Entonces, considerando un promedio original de $4.8$ se requirieron $18$ ratings de 5 puntos para volver a dicho promedio original, sin contar la posibilidad de que otro usuario le haya asignado otro $1$.

#### Qué se puede hacer?

* Mantener los ratings, pero considerar también la variabilidad en los procesos de rateo de los usuarios.

Debería funcionar de la misma forma que las métricas que utilizamos hasta ahora, con la diferencia que el rating o score final dado a un elemento ahora considera cuánto se desvía el rating asignado respecto al rating promedio asignado por el usuario. Dado el promedio anterior $Sc_{D}$, el rating promedio del usuario $AVG(R_{p})$ y el rating del elemento actual $R_{p,d}$, el rating debería ser actualizado como:

$$Sc_{D} = Sc_{D} + \frac{AVG(R_{p}) - R_{p,d}}{100}$$

Este enfoque solo soluciona los problemas de escala entre los distintos usuarios pero no soluciona la penalización que el promedio da a los ratings bajos que los usuarios dan a elementos con promedios altos.

* Utilizar valoraciones binarias (solo positivo o negativo).

Como el actualmente utilizado por Netflix. Permite hacer más sencillo el problema de asignar ratings.


## Recomendación basada en contenido

En este tipo de recomendaciones, no se necesita contar con una gran cantidad de usuarios, sino contar con una descripción de los elementos y los perfiles de los usuarios, los cuales deben matchear. 

### Ventajas

*   No se necesita gran cantidad de usuarios para proveer recomendaciones confiables.
*   Pueden darse recomendaciones solo considerando las características de los elementos.

### Desventajas

* Las descripciones de los elementos pueden no ser sencillas de construir.
* Los usuarios no utilizan tags de forma homogénea.


Una forma muy sencilla de realizar recomendaciones basadas en contenido es hacer uso de los tags que les fueran asignados a los elementos.

Para esta primera parte, vamos a usar otro [dataset](https://raw.githubusercontent.com/codeheroku/Introduction-to-Machine-Learning/master/Building%20a%20Movie%20Recommendation%20Engine/movie_dataset.csv) que también tiene información de películas, género, director, tags, actores y demás; y una libría para hacer más fácil los cálculos.

In [0]:
# como veníamos haciendo antes, vamos a cargar el dataset
url = 'https://raw.githubusercontent.com/codeheroku/Introduction-to-Machine-Learning/master/Building%20a%20Movie%20Recommendation%20Engine/movie_dataset.csv'
movies_2 = pd.read_csv(url)

movies_2.head()


Al visualizar el dataset se ve que existe información extra acerca de las películas, la cual no es necesaria. Entonces, vamos solo a seleccionar los tags o `keywords`, el `cast`, los `genres` y el `director`, es decir, todo aquello que pueda conseridarse como `contenido` de las películas. 

Para ello, vamos a seleccionar esos features y crear una función que nos haga el `join` de todos esos campos

In [0]:
features = ['keywords','cast','genres','director']

In [0]:
def combine_features(row):
    return row['keywords']+" "+row['cast']+" "+row['genres']+" "+row['director']

Ahora necesitamos invocar a la función `combine_feature` a cada fila del data frame. Pero antes de eso, vamos a "limpiar"  y pre-procesar los datos. Para ello vamos a reemplazar los NaN por espacios en blanco.


In [0]:

for feature in features:
    movies_2[feature] = movies_2[feature].fillna('') #filling all NaNs with blank string

movies_2['combined_features'] = movies_2.apply(combine_features,axis=1) #applying combined_features() method over each rows of dataframe and storing the combined string in "combined_features" column

movies_2[['original_title','combined_features']].head()

Una vez que se tienen los `String` combinados, se puede utilizar un `CountVectorizer()` para obtener la matriz de frecuencias.

Now that we have obtained the combined strings, we can now feed these strings to a CountVectorizer() object for getting the count matrix.

In [0]:
from sklearn.feature_extraction.text import CountVectorizer

cv = CountVectorizer() #creating new CountVectorizer() object
count_matrix = cv.fit_transform(movies_2['combined_features']) #feeding combined strings(movie contents) to CountVectorizer() object


Vamos a calcular el `cosine similarity`:

In [0]:
from sklearn.metrics.pairwise import cosine_similarity

cosine_sim = cosine_similarity(count_matrix) # nos retorna un numpy.ndarray

cosine_sim.shape


El próximo paso es obtener los títulos de las películas en las que los usuarios se encuentran actualmente interesados. Luego vamos a encontar el índice de la película y relacionarlo con la semejanza calculada. En consecuencia, vamos a obtener la semejanza para con todas las otras películas de una película dada. Dichas semejazas son enumeradas y transformadas en una tupla que contiene `(movie id, similarity score)`.


In [0]:
movie_user_likes = "Harry Potter and the Order of the Phoenix"
movie_index = movies_2[movies_2['original_title'] == movie_user_likes]["index"].values[0]
similar_movies = list(enumerate(cosine_sim[movie_index])) #accedemos a la fila correspondiente a una película dada para obtener sus semejanzas con el resto.
print(similar_movies)

Ahora viene la parte más importante. Como veníamos haciendo antes, hay que ordenar la lista de películas similares de forma descendente, descartando si misma de la lista.


In [0]:
sorted_similar_movies = sorted(similar_movies,key=lambda x:x[1],reverse=True)[1:]

sorted_similar_movies = sorted_similar_movies[0:10] # nos interesan solo las 10 primeras

x = [p[0] for p in sorted_similar_movies] # obtenemos los index de las películas

movie_titles = movies_2[movies_2['index'].isin(x)]

movie_titles['original_title']





### Volviendo al dataset que estábamos usando...

El dataset que tenemos cargado, no tiene asociado contenido para las diferentes películas ni tags ni reviews, así que vamos a cargar otro archivo con tags para cada película.

In [0]:
url = 'https://raw.githubusercontent.com/tommantonela/sistemasRecomendacion2019/master/ml-20m/tags.csv'
tags = pd.read_csv(url, sep=',', names=['user_id','movie_id','tag','timestamp'],header=0)

url = 'https://raw.githubusercontent.com/tommantonela/sistemasRecomendacion2019/master/ml-20m/movies.csv'
full_movies = pd.read_csv(url, sep=',', names=['movie_id','movie_title','genres'],header=0)
full_movies = full_movies.drop('genres', axis=1) # tenemos que cargar los títulos porque los ids de las películas difieren de dataset en dataset

tags = pd.merge(tags,full_movies, on='movie_id')

print(str(tags.shape))

tags.head()

Como este dataset tiene muchas más películas que con el que veníamos trabajando, vamos a eliminar todos los tags pertenecientes a las películas que no tenemos.

In [0]:
titles = df['movie_title'] #acá nos quedamos con los títulos de las pelis con una cantidad de reviews mayores al 1er cuartil (lo que habíamos calculado para los otros tipos de recomendaciones)

tags = tags[tags['movie_title'].isin(titles)]
print(str(tags.shape))

tags.head()

Ahora ya tenemos la información de los tags para las diferentes películas que ya teníamos. En el dataframe `tags` cada fila representa una asignación de `tag` a una película.
Vamos a poder:

*   Crear descripciones de las películas en base a todos los tags que les hayan sido asignados. Con esto se puede determinar la semejanza entre las películas.
*   Crear perfiles de los usuarios con todos los tags que haya asignado a las películas. Con esto luego se puede determinar su semejanza con la descripción de otras películas.



In [0]:
# creación de las descripciones de las películas

tag_matrix = tags.pivot_table(index='user_id', columns='movie_title', values='tag',aggfunc='first')
tag_matrix.head()

# tag_matrix['101 Dalmatians (1996)'].dropna() # ejemplo para ver los tags asociados a una película determinada

In [0]:
# creación de los perfiles de usuario
#user_tag_matrix = tags.pivot_table(index='movie_title', columns='user_id', values='tag',aggfunc='first')

# user_tag_matrix[65].dropna() # ejemplo para ver el perfil de un usuario en particular

Ahora, ya podemos calcular las semejanzas con los perfiles!

#### Tarea!

Considerando el dataframe original:

*   Armar los perfiles de los usuarios considerando los tags que haya asignado a películas que haya dado una mala calificación ($<2$). Vamos a suponer que esas películas no son del agrado de los usuarios.
  *  Para esto pueden probar de mergear los `data frames` y filtrar cómo ya hicimos antes.
*   Dado un usuario cualquiera, encontrar el ranking de las 10 películas más disimilares que no haya rateado.
  * Dado un usuario, pueden seleccionar su perfil.
  * Seleccionar las películas que no haya rateado.
  * Calcular la semejanza entre esas películas y su perfil.
  * Ordenar y obtener las 10 más disimilares.



In [0]:
# TODO!